# Preliminary Predictive Modeling

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

## Logistic Regression